# Parse and prepare a dataset of abc music notations

Download the [Nottingham Dataset](https://github.com/jukedeck/nottingham-dataset) or this [dataset of abc music notation from Henrik Norbeck ](http://norbeck.nu/abc/download.asp) select the 'one big zip file (549 kilobytes).' at the end of the page. 

If we use the Henrik Norbeck DS the first thing we are going to do is parse all the files and concatenate the text in one 'big' text file.

We will train our model using Char-RNN for TF, you can clone it from [https://github.com/sherjilozair/char-rnn-tensorflow](https://github.com/sherjilozair/char-rnn-tensorflow)

## ABC Notations

We will need some software to work with `abc` and `mid` files, you can install by using on Ubuntu:

```
$ sudo apt-get install abcmidi timidity
```

On Mac:


```
$ brew install abcmidi timidity
```

For mac or pc user you can also install [easy abc](https://www.nilsliberg.se/ksp/easyabc/) to read the files

Here’s a simple example:

```
X: 1
T:"Hello world in abc notation"
M:4/4
K:C
"Am" C, D, E, F,|"F" G, A, B, C|"C"D E F G|"G" A B e c
```

To test the installation we can listen to this by saving the above snippet into a `hello.abc` file and running (Mac and Ubuntu):

```
$ abc2midi hello.abc -o hello.mid && timidity hello.mid
```

In [4]:
import os

# input_folder_fp = '/home/gu-ma/Downloads/hn201809'
input_folder_fp = r'C:\Users\simon\Documents\GitHub\ComputationalPerceptionExtended\notebooks\datasets\nottinghamDataset\ABC_cleaned'
abc_raw_txt = ''
abc_all_txt = ''

# Parse all files in the input folders
for root, subdirs, files in os.walk(input_folder_fp):
    print(root)
    for filename in files:
        file_path = os.path.join(root, filename)
        print('\t- %s ' % filename)
        if filename.lower().endswith('.abc'):
            with open(file_path, 'r') as f:
                abc_raw_txt += f.read()

print('\nabc_raw_txt:\n--\n' + abc_raw_txt[:1000])

C:\Users\simon\Documents\GitHub\ComputationalPerceptionExtended\notebooks\datasets\nottinghamDataset\ABC_cleaned
	- abc_all.txt 
	- abc_headers.txt 
	- abc_raw.txt 
	- ashover.abc 
	- hpps.abc 
	- jigs.abc 
	- morris.abc 
	- playford.abc 
	- reelsa-c.abc 
	- reelsd-g.abc 
	- reelsh-l.abc 
	- reelsm-q.abc 
	- reelsr-t.abc 
	- reelsu-z.abc 
	- slip.abc 
	- waltzes.abc 
	- xmas.abc 

abc_raw_txt:
--

X: 1
T:A and A's Waltz
% Nottingham Music Database
S:Mick Peat
M:3/4
L:1/4
K:G
e|:"G"d2B|"D"A3/2B/2c|"G"B2G|"D"A2e|"G"d2B|"D"A3/2B/2c|
M:2/4
"F"B=F|
M:3/4
"G"G2e:||:
"C"g2e|"Bb"=f2d|"F"c2A|=F2e|"C"g2e|"Bb"=f2d|
M:2/4
"F"cA|
M:3/4
 [1 "G"G2e:| [2"G"G2z||


X: 2
T:Barry's Favourite
% Nottingham Music Database
S:Mick Peat
M:2/2
K:D
A2|:"D"a3/2b/2a3/2g/2 f2(3def|"Em"g3/2a/2g3/2f/2 "A"e2A2|"D"f3/2g/2f3/2e/2 d2f2\
|"Em"B3/2c/2d3/2e/2 "A"c2A2|
"D"a3/2b/2a3/2g/2 f2(3def|"Em"g3/2a/2g3/2f/2 "A"e2A2|\
"D"f3/2g/2f3/2e/2 d3/2e/2f3/2A/2|"G"B3/2d/2"A"d3/2c/2 "D"d2A2:|
|:"G"B3/2A/2B3/2g/2 "D"d2A2|"Em"e3/2d/2

Then we remove the 'unecessary' parts, clean up the text

In [5]:
import re

# Helper function to extract (and delete) chunks of text from abc_raw_text
def extract_text(regex, txt, delete):
    output = ''
    # extract the text
    for result in re.findall(regex, txt, re.S):
        output += result + "\n"
    # delete from the original file
    if delete:        
        global abc_raw_txt
        abc_raw_txt = (re.sub(regex, '', abc_raw_txt, flags=re.S))
    # remove empty lines
    abc_raw_txt = ''.join([s for s in abc_raw_txt.strip().splitlines(True) if s.strip()])
    return output

# Helper function to delete selected lines from a text
def delete_lines(regex, txt):
    txt = (re.sub(regex, '', txt, flags=re.S))
    txt = ''.join([s for s in txt.strip().splitlines(True) if s.strip()])
    return txt

# Extract intro text
useless_txt = extract_text(r'(This file.*?- Questions?.[^\n]*)', abc_raw_txt, True)

# Save the file without the intro text
abc_all_txt = abc_raw_txt

# Delete 'comments'
abc_raw_txt = delete_lines(r'".[^\n]*', abc_raw_txt)
# Delete Lyrics
abc_raw_txt = delete_lines(r'%.[^\n]*', abc_raw_txt)
# Delete some more comments
abc_raw_txt = delete_lines(r'W:.[^\n]*', abc_raw_txt)

# Extract headers
abc_headers_txt = extract_text(r'(X:.*?K:.[^\n]*)', abc_raw_txt, True)

print('\nabc_raw_txt:\n--\n' + abc_raw_txt[:1000])
print('\nabc_headers_txt:\n--\n' + abc_headers_txt[:1000])


abc_raw_txt:
--
e|:
M:2/4
M:3/4
M:2/4
M:3/4
 [1 
A2|:
|
|:
|:
|:
M:6/8
K:D
|:
|:
d2|: || |:
A/2|: || |:
[1
f|: || |:
|:
|:
G/2C/2_E/2G/2 C/2E/2G/2E/2|
A/2D/2F/2A/2 D/2F/2A/2F/2|
^F/2G/2e _ee|[1
|
[Ac][GB][FA]|
[Bd][G/2B/2][A/2c/2][Bd]|
[B3g3]|
[ce][Bd][Ac]|
K:G
P:A
^G/2A/2^A/2
P:B
|:
|
[1
P:C
K:Cm
|:
e|:
|:
P:A
d
P:B
|:
P:C
|:
|
G|:
f/2|:
K:D
|:
c/2B/2G/2B/2 c/2B/2c/2d/2|c/2B/2G/2B/2 c/2B/2c/2d/2|\
e/2A/2(3c/2B/2A/2 f/2B/2(3d/2c/2B/2|e/2E/2(3G/2F/2E/2 e2:|
K:A
K:A
P:A
|:
P:B
[ac]|:
 [2
|:
G/2|:
K:G
|: || |:
A/2|:
[1 
|
B3/2c/2|
(3BAG|:
e3/2A/2G3/2A/2 E3/2B/2A3/2B/2|
 [1
G/2F/2|:
G2A |
|:
B/2c/2 |:
|
|:A/4B/4|
A|:
=cBA Ade|fgf ed=c|[1
K:A
|:
f/2g/2|
E/2C/2 
|:
(3a/4g/4f/4|:
|:
B|:
|:
P:A
d3:|\
P:B
K:A
a3||
E3|
|
A3/2d/2|:
K:F
K:F
K:D
A3/2d/2:|
d2 B3/2d/2|
[1
[2
K:A
|:
B,C ^C|:
BA2^A|
BG E=F|
P:A
[e3g3]|
[D2F2][CE]|
P:B
[C2A2][DB]|[E2c2][DB]|
[D2B2][Ec]|[D2B2][CA]|[B,2G2][CA]|[B,2G2][A,F]|[G,2E2][G,D]|
P:A
D\
P:B
|:
P:C
e|: || |:
P:A
DE \
P:B
|:
P:C
|:
P:A
d2|
P:B
G2|
P:A
(3fg^g|
P:B
e2

In [6]:
print(len(abc_raw_txt))
print(len(abc_headers_txt))

22260
63186


Once we have what we need we can save the file to disk

In [7]:
output_raw_fp = os.path.join(input_folder_fp, 'abc_raw.txt')
output_all_fp =  os.path.join(input_folder_fp, 'abc_all.txt')
output_header_fp =  os.path.join(input_folder_fp, 'abc_headers.txt')

with open(output_raw_fp, 'w') as f:
    f.write(abc_raw_txt)
    
with open(output_all_fp, 'w') as f:
    f.write(abc_all_txt)
    
with open(output_header_fp, 'w') as f:
    f.write(abc_headers_txt)

Now that we have our input text file ready we can run it through our RNN, we will use char-rnn for tensorflow, you can download it and install it from [here](https://github.com/sherjilozair/char-rnn-tensorflow) 

In [11]:
import shutil
import subprocess

charrnn_folder_fp = '/home/gu-ma/Documents/Projects/201809-HSLU-COMPPX/References/char-rnn-tensorflow'

# We try with the full text first
# shutil.move(output_all_fp, os.path.join(charrnn_folder_fp, 'data', 'abc', 'input.txt'))
shutil.copy(output_raw_fp, os.path.join(input_folder_fp, 'input.txt'))

'C:\\Users\\simon\\Documents\\GitHub\\ComputationalPerceptionExtended\\notebooks\\datasets\\nottinghamDataset\\ABC_cleaned\\input.txt'

Go to the directory and run the training. Rename clean_all.txt to input.txt and move it to the downloaded RNN repo data/abc/ folder

In [1]:
# on the terminal
# python train.py --data_dir=./data/abc/


create a new soundfile

In [17]:
# sample.py --prime "X:"

then replace \n with an actual linebreak (regular expression find \\n replace with \n)